In [1]:
from sklearn.preprocessing import MultiLabelBinarizer
from scrape_funcs import zip_files, source_dir, ZipFile,BeautifulSoup, merge_dicts, get_all_data_from_site
import numpy as np
import pandas as pd
import re

data_dict ={}
for i, file in enumerate(zip_files(source_dir)):
    print(i, end='\r')
    file_dir = f'{source_dir}\\{file}'
    with ZipFile(file=file_dir) as zip_file:
        bs = zip_file.read(zip_file.namelist()[0]).decode('utf8')
        soup = BeautifulSoup(bs, "html.parser")
        new_dict = get_all_data_from_site(soup,file[:-4])
        data_dict = new_dict if not data_dict else merge_dicts(data_dict,new_dict)
        if i == 25:
            break


In [ ]:
for i, file in enumerate(zip_files(source_dir)):
    print(file)

In [17]:
from scrape_funcs import *
get_housing_links()

1717      sachsenhausen.6646560
1814       nordend-west.9064498
1815       preungesheim.8692997
1816           eschborn.4714758
1817    bahnhofsviertel.8597515
                 ...           
1894         roedelheim.9041685
1895       nordend-west.9062594
1896          niederrad.9095992
1897        westend-sud.8081008
1898      sachsenhausen.7184387
Name: Links, Length: 86, dtype: object

In [2]:
import pandas as pd

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

In [3]:
df = pd.DataFrame(data_dict,dtype=str)
df = df.apply(lambda x: x.astype(str).str.lower())

In [4]:
def replace_umlauts(string):
    umlautDictionary = {u'Ä': 'Ae',
                    u'Ö': 'Oe',
                    u'Ü': 'Ue',
                    u'ä': 'ae',
                    u'ö': 'oe',
                    u'ü': 'ue',
                    u'ß': 'ss',
                    u':': '',
                    }
    umap = {ord(key):str(val) for key, val in umlautDictionary.items()}
    return string.translate(umap)

In [5]:
# rename columns
columns = [ re.sub('[^\w]', '_', replace_umlauts(x).lower().strip()) for x in df.columns]
df.columns = columns

# remove common words from cols

remove_list = ["m²","€",r"n.a.","none","(",")", ". og","minuten zu fuß entfernt","minute zu fuß entfernt", "schneller als", "mbit/s"]
for col in list(df.columns):
    for item in remove_list:
        df[col] = df[col].str.replace(item,"", regex=False)

df["wifi"] = df["wifi"].str.replace("  ", " ", regex=False)
df["wifi"] = df["wifi"].str.replace("wlan\s", "wlan", regex=True)
df["wifi"] = df["wifi"].str.replace("-", "_", regex=False).str.strip()

#remove individual words from col
df["personen"] = df["weiblich"]+df["maennlich"]+df["gesucht"]

df["strasse"] = df.adresse.str.extract(pat="(.*)\d\d\d\d\d")
df["strasse"] = df.strasse.str.replace("str\.","strasse", regex =False)
df["strasse"] = df.strasse.str.replace("str ","strasse")
df["strasse"] = df.strasse.str.replace("strasse","strasse")
df["strasse"] = df.strasse.str.replace("[^\w\d]","", regex =True)
df["strasse"] = df.strasse.str.replace("[0-9]+","", regex =True)
df["strasse"] = df.strasse.str.replace("ß","ss")
# Remove rare streets
#df.loc[~df["strasse"].isin(list(pd.DataFrame(df.strasse.value_counts()).query("strasse > 10").index)), "strasse"] = "no_info_or_rare"

# Vermutlich ist möbliert, teilmöbliert = teilmöbliert
df["bed"]      = df.bed.str.replace("möbliert, teilmöbliert","teilmöbliert")
df["geschlecht"] = df["geschlecht"].str.replace("egalegal","egal")
df["folder"] = df["folder"].str.replace("haustiere erlaubt","haustiere")
one_hot_cols = ["folder",'layers','wifi',"shower"]
df_dict = {elem : pd.DataFrame() for elem in one_hot_cols}

distinct_value_list = []
for col in one_hot_cols:
    values = list(set([val.strip() for sublist in df[col].dropna().str.split(",").tolist() for val in sublist]))
    distinct_value_list.append(values)
    df[col] = df[col].str.replace(",","")
    df[col] = df[col].str.split(" ").str[:]

mlb = MultiLabelBinarizer()
for i, col in enumerate(df_dict.keys()):
    mlb.fit_transform(df[col])
    df_dict[col] = pd.DataFrame(mlb.fit_transform(df[col]),columns=mlb.classes_, index=df.link)

# Reverse binarization for wlan values
cols =list(df_dict["wifi"].filter(regex=("wlan.*")).columns)
wifi_df = df_dict["wifi"][cols+[""]].copy()
df_dict["wifi"].drop(columns=list(cols), inplace=True)
df_dict["wifi"] = df_dict["wifi"].join(pd.DataFrame(wifi_df.idxmax(1),columns = ["wlan"]))

df = df.set_index("link")
for col in df_dict:
    df = df.join(df_dict[col]).drop("", axis=1)

df = df.drop(columns= one_hot_cols)
conditions = [
    (df["frei_ab"] == ""),
    ((df["frei_ab"] != "") & (df["frei_bis"] != "")),
    ((df["frei_ab"] != "") & (df["frei_bis"] == ""))]
choices = ['inaktiv', 'befristet', 'unbefristet']
df["status"] = np.select(conditions, choices)

df["direct_link"]= "https://www.wg-gesucht.de/wg-zimmer-in-Frankfurt-am-Main-" + df.index + ".html"

df["dauer"] = pd.to_datetime(df.frei_bis, format='%d.%m.%Y', errors='coerce') - pd.to_datetime(df.frei_ab, format='%d.%m.%Y', errors='coerce')
df['dauer'] = df['dauer'] / np.timedelta64(1, 'D')
df['dauer'].fillna(0, inplace=True)

df["wohnung"] = df["wohnung"].str.replace("\.","", regex= False)
df["m2_pro_pers"] = pd.to_numeric(df['wohnung'], errors='coerce')/ pd.to_numeric(df['personen'], errors='coerce')
df["baujahr"] = df.baujahr.str.extract(pat="baujahr (\d\d\d\d)")
df["plz"] = df.adresse.str.extract("(\d\d\d\d\d)")

num_cols = ['wohnung', 'zimmergroesse', 'sonstige', 'nebenkosten', 'miete', 'gesamtmiete', 'bus', 'maennlich',
 'personen', 'weiblich', 'kaution', 'abloesevereinbarung', 'personen', 'bus', 'baujahr',"taeglich"]
for col in num_cols:
    df[col] = df[col].astype(str)
    df[col] = df[col].str.extract('(\d+)', expand=False)
    df[col] = df[col].astype(float)


In [6]:
# Filter outliers
df = df.query("150 < gesamtmiete < 1200 and 6 < zimmergroesse < 60")

In [7]:
df.head(1)

,wohnung,zimmergroesse,sonstige,nebenkosten,miete,gesamtmiete,kaution,abloesevereinbarung,frei_ab,frei_bis,weiblich,maennlich,gesucht,zwischen,geschlecht,bed,city,monitor,car,silverware,office,bus,adresse,taeglich,baujahr,rauchen,zweck_wg,keine_zweck_wg,berufstaetigen_wg,gemischte_wg,studenten_wg,frauen_wg,azubi_wg,personen,strasse,aufzug,balkon,fahrradkeller,garten,gartenmitbenutzung,haustiere,keller,spülmaschine,terrasse,waschmaschine,dielen,fliesen,fußbodenheizung,laminat,parkett,teppich,dsl,flatrate,wlan,badewanne,dusche,status,direct_link,dauer,m2_pro_pers,plz
link,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
bahnhofsviertel.6048855,NaN,18.0,NaN,150.0,500.0,650.0,650.0,NaN,01.03.2022,,1.0,2.0,1,,egal,möbliert,mehrfamilienhaus,,schlechte parkmöglichkeiten,,3,3.0,elbestraße 3.ogre 60329 frankfurt am main bahnhofsviertel,0.0,1960.0,nichtraucher,1,0,1,1,1,0,0,121.0,elbestrasseogre,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,wlan,0,1,unbefristet,https://www.wg-gesucht.de/wg-zimmer-in-Frankfurt-am-Main-bahnhofsviertel.6048855.html,0.0,NaN,60329


In [8]:
# Replace na's
#keine angabe
feat_list1 = ["city", "silverware", "bed", "car", "monitor", "strasse","plz"]
#median
feat_list2 = ["wohnung","bus","personen","m2_pro_pers"]
# null setzen
feat_list3 = ["baujahr","abloesevereinbarung"]

for feat in feat_list1:
    df[feat].fillna("no_info_or_rare", inplace=True)
    df[feat] = df[feat].replace(r'^\s*$', "no_info_or_rare", regex=True)

for feat in feat_list2:
    df[feat].fillna(df[feat].median(), inplace=True)
    df[feat] = df[feat].replace(r'^\s*$', df[feat].median(), regex=True)

for feat in feat_list3:
    df[feat].fillna(0, inplace=True)
    df[feat] = df[feat].replace(r'^\s*$', 0, regex=True)

In [9]:
df.head()

,wohnung,zimmergroesse,sonstige,nebenkosten,miete,gesamtmiete,kaution,abloesevereinbarung,frei_ab,frei_bis,weiblich,maennlich,gesucht,zwischen,geschlecht,bed,city,monitor,car,silverware,office,bus,adresse,taeglich,baujahr,rauchen,zweck_wg,keine_zweck_wg,berufstaetigen_wg,gemischte_wg,studenten_wg,frauen_wg,azubi_wg,personen,strasse,aufzug,balkon,fahrradkeller,garten,gartenmitbenutzung,haustiere,keller,spülmaschine,terrasse,waschmaschine,dielen,fliesen,fußbodenheizung,laminat,parkett,teppich,dsl,flatrate,wlan,badewanne,dusche,status,direct_link,dauer,m2_pro_pers,plz
link,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
bahnhofsviertel.6048855,115.0,18.0,NaN,150.0,500.0,650.0,650.0,0.0,01.03.2022,,1.0,2.0,1,,egal,möbliert,mehrfamilienhaus,no_info_or_rare,schlechte parkmöglichkeiten,no_info_or_rare,3,3.0,elbestraße 3.ogre 60329 frankfurt am main bahnhofsviertel,0.0,1960.0,nichtraucher,1,0,1,1,1,0,0,121.0,elbestrasseogre,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,wlan,0,1,unbefristet,https://www.wg-gesucht.de/wg-zimmer-in-Frankfurt-am-Main-bahnhofsviertel.6048855.html,0.0,0.841584,60329
bahnhofsviertel.5977052,300.0,15.0,NaN,150.0,450.0,600.0,600.0,0.0,26.02.2022,,4.0,3.0,3,,egal,möbliert,mehrfamilienhaus,kabel,schlechte parkmöglichkeiten,no_info_or_rare,3,3.0,taunusstraße hh.3r 60329 frankfurt am main bahnhofsviertel,0.0,1975.0,nichtraucher,1,0,1,1,1,0,0,433.0,taunusstrassehhr,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,wlan,0,1,unbefristet,https://www.wg-gesucht.de/wg-zimmer-in-Frankfurt-am-Main-bahnhofsviertel.5977052.html,0.0,0.692841,60329
bahnhofsviertel.5840538,130.0,15.0,NaN,150.0,570.0,720.0,720.0,0.0,26.02.2022,,2.0,3.0,1,,egal,möbliert,mehrfamilienhaus,kabel,schlechte parkmöglichkeiten,no_info_or_rare,höher als 5,3.0,elbestraße 6.og 60329 frankfurt am main bahnhofsviertel,0.0,1960.0,keine_angabe,1,0,1,1,1,0,0,231.0,elbestrasseog,1,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,1,1,wlan,0,1,unbefristet,https://www.wg-gesucht.de/wg-zimmer-in-Frankfurt-am-Main-bahnhofsviertel.5840538.html,0.0,0.562771,60329
bad-vilbel.7778992,140.0,15.0,NaN,NaN,395.0,395.0,395.0,0.0,01.04.2022,30.09.2022,1.0,2.0,1,egal,egal,möbliert,reihenhaus,no_info_or_rare,gute parkmöglichkeiten,no_info_or_rare,1,4.0,nidderring 0 61118 frankfurt am main bad vilbel,0.0,0.0,nichtraucher,1,1,1,1,1,0,0,121.0,nidderring,0,1,0,1,1,0,0,1,1,1,0,1,0,0,0,1,0,0,,0,1,befristet,https://www.wg-gesucht.de/wg-zimmer-in-Frankfurt-am-Main-bad-vilbel.7778992.html,182.0,1.157025,61118
bad-vilbel.7778855,140.0,18.0,NaN,NaN,410.0,410.0,410.0,0.0,01.04.2022,30.09.2022,1.0,2.0,1,egal,egal,möbliert,reihenhaus,no_info_or_rare,gute parkmöglichkeiten,no_info_or_rare,1,4.0,nidderring 0 61118 frankfurt am main bad vilbel,0.0,0.0,nichtraucher,1,1,1,1,1,0,0,121.0,nidderring,0,1,0,1,1,0,0,1,1,1,0,1,0,0,0,1,0,0,,0,1,befristet,https://www.wg-gesucht.de/wg-zimmer-in-Frankfurt-am-Main-bad-vilbel.7778855.html,182.0,1.157025,61118


## BS test

In [13]:
from extract_functions import *

In [14]:
full_link = "https://www.wg-gesucht.de/wg-zimmer-in-Frankfurt-am-Main-westend-sud.9129828.html"
stem = full_link[:57]
link = full_link[57:]
bs = get_bs_from_http(stem+link)

In [16]:
my_list = get_text_from_html(bs,"ul-detailed-view-datasheet print_text_left")
my_list = [x.strip() for x in my_list]
my_list

['20m²\n                    \n                    Zimmer in 3er WG \n                \n\n                                            Wohnungsgröße: 90m²\n                                    \n\n                    3er WG\n                                                                        (\n                                                            1\n                                                                    Frau\n                                                            \n                                                            und\n                            \n                                                            1\n                                                                    Mann\n                                                            \n                            \n                                                        )\n                                                            \n\n\n\n                        Business-WG, Berufstätigen-WG

In [ ]:

get_all_data_from_site(bs=bs, link=link)

In [22]:
item = "headline headline-detailed-view-title"

In [48]:
people_list = bs.find(class_=item).find_all(class_="vis")

In [50]:
str(people_list)

'[<img alt="weiblich" class="vis" height="17" src="/img/wgw.gif" style="margin-left:0px;" width="10"/>, <img alt="Malmännliche" class="vis" height="17" src="/img/wgm.gif" style="margin-left:0px;" width="10"/>, <img alt="Mitbewohnerin oder Mitbwohner gesucht" class="vis" height="17" src="/img/wgeg.gif" style="margin-left:0px;" width="10"/>]'

In [53]:
str(people_list).count("vis")

3

In [54]:
people_list

[<img alt="weiblich" class="vis" height="17" src="/img/wgw.gif" style="margin-left:0px;" width="10"/>,
 <img alt="Malmännliche" class="vis" height="17" src="/img/wgm.gif" style="margin-left:0px;" width="10"/>,
 <img alt="Mitbewohnerin oder Mitbwohner gesucht" class="vis" height="17" src="/img/wgeg.gif" style="margin-left:0px;" width="10"/>]

In [7]:
"mdi mdi-shower mdi-36px noprint".split("-")[1].split()[0]

'shower'

In [3]:
from scrape_funcs import *
zip_my_files()